In [1]:
import numpy as np
import re
import pickle
class Preprocessing:

    @staticmethod
    def read_dataset(file):

        letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
                     'n','o','p','q','r','s','t','u','v','w','x','y','z',' ']

        # Open raw file
        with open(file, 'r') as f:
            raw_text = f.readlines()

        # Transform each line into lower
        raw_text = [line.lower() for line in raw_text ]
        
        # Create a string which contains the entire text
        text_string = ''
        for line in raw_text:
            text_string += line.strip()

# Create an array by char
        text = list()
        for char in text_string:
            text.append(char)

# Remove all symbosl and just keep letters
        text = [char for char in text  if char in letters]

        return text

    @staticmethod
    def create_dictionary(text):

        char_to_idx = dict()
        idx_to_char = dict()
        
        idx = 0
        for char in text:
            if char not in char_to_idx.keys():
                char_to_idx[char] = idx
                idx_to_char[idx] = char
                idx += 1
        
        return char_to_idx, idx_to_char
        
    @staticmethod
    def build_sequences_target(text, char_to_idx, window):
        
        x = list()
        y = list()

        for i in range(len(text)):
            try:
                # Get window of chars from text
                # Then, transform it into its idx representation
                sequence = text[i:i+window]
                sequence = [char_to_idx[char] for char in sequence]
                
                # Get char target
                # Then, transfrom it into its idx representation
                target = text[i+window]
                target = char_to_idx[target]
                
                # Save sequences and targets
                x.append(sequence)
                y.append(target)
            except:
                pass
        
        x = np.array(x)
        y = np.array(y)

        return x, y

In [2]:
import argparse

def parameter_parser():

    parser = argparse.ArgumentParser(description = "Text Generation")
    parser.add_argument('-f')
    parser.add_argument("--epochs", dest="num_epochs", type=int, default=100)
    parser.add_argument("--learning_rate", dest="learning_rate", type=float, default=0.005)
    parser.add_argument("--hidden_dim", dest="hidden_dim", type=int, default=6)
    parser.add_argument("--batch_size", dest="batch_size", type=int, default=2)
    parser.add_argument("--window", dest="window", type=int, default=5)
    parser.add_argument("--load_model", dest="load_model", type=bool, default=False)
    parser.add_argument("--model", dest="model", type=str, default='weights/textGenerator.pt')
 
    return parser.parse_args()

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextGenerator(nn.ModuleList):
    def __init__(self, args, vocab_size):
        super(TextGenerator, self).__init__()

        self.batch_size = args.batch_size
        self.hidden_dim = args.hidden_dim
        self.input_size = vocab_size
        self.num_classes = vocab_size
        self.sequence_len = args.window

        # Dropout
        self.dropout = nn.Dropout(0.25)

        # Embedding layer
        self.embedding = nn.Embedding(self.input_size, self.hidden_dim, padding_idx=0)

        # Bi-LSTM
        # Forward and backward
        self.lstm_cell_forward = nn.LSTMCell(self.hidden_dim, self.hidden_dim)
        self.lstm_cell_backward = nn.LSTMCell(self.hidden_dim, self.hidden_dim)

        # LSTM layer
        self.lstm_cell = nn.LSTMCell(self.hidden_dim * 2, self.hidden_dim * 2)

        # Linear layer
        self.linear = nn.Linear(self.hidden_dim * 2, self.num_classes)

    def forward(self, x):

        # Bi-LSTM
        # hs = [batch_size x hidden_size]
        # cs = [batch_size x hidden_size]
        hs_forward = torch.zeros(x.size(0), self.hidden_dim)
        cs_forward = torch.zeros(x.size(0), self.hidden_dim)
        hs_backward = torch.zeros(x.size(0), self.hidden_dim)
        cs_backward = torch.zeros(x.size(0), self.hidden_dim)
        
        # LSTM
        # hs = [batch_size x (hidden_size * 2)]
        # cs = [batch_size x (hidden_size * 2)]
        hs_lstm = torch.zeros(x.size(0), self.hidden_dim * 2)
        cs_lstm = torch.zeros(x.size(0), self.hidden_dim * 2)

        # Weights initialization
        torch.nn.init.kaiming_normal_(hs_forward)
        torch.nn.init.kaiming_normal_(cs_forward)
        torch.nn.init.kaiming_normal_(hs_backward)
        torch.nn.init.kaiming_normal_(cs_backward)
        torch.nn.init.kaiming_normal_(hs_lstm)
        torch.nn.init.kaiming_normal_(cs_lstm)


        # From idx to embedding
        out = self.embedding(x)

        # Prepare the shape for LSTM Cells
        out = out.view(self.sequence_len, x.size(0), -1)
        
        forward = []
        backward = []
        
        # Unfolding Bi-LSTM
        # Forward
        for i in range(self.sequence_len):
            hs_forward, cs_forward = self.lstm_cell_forward(out[i], (hs_forward, cs_forward))
        forward.append(hs_forward)

        # Backward
        for i in reversed(range(self.sequence_len)):
            hs_backward, cs_backward = self.lstm_cell_backward(out[i], (hs_backward, cs_backward))
            backward.append(hs_backward)

        # LSTM
        for fwd, bwd in zip(forward, backward):
            input_tensor = torch.cat((fwd, bwd), 1)
            hs_lstm, cs_lstm = self.lstm_cell(input_tensor, (hs_lstm, cs_lstm))

        # Last hidden state is passed through a linear layer
        out = self.linear(hs_lstm)

        return out


C:\ProgramData\Anaconda3\envs\ML_cln\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# from txtge import TextGenerator
# from mod import Preprocessing
# from mod import parameter_parser

class Execution:

    def __init__(self, args):
        self.file = 'data/dataset.txt'
        self.window = args.window
        self.batch_size = args.batch_size
        self.learning_rate = args.learning_rate
        self.num_epochs = args.num_epochs
        
        self.targets = None
        self.sequences = None
        self.vocab_size = None
        self.char_to_idx = None
        self.idx_to_char = None

    def prepare_data(self):

        # Initialize preprocessor object
        preprocessing = Preprocessing()

        # The 'file' is loaded and split by char
        text = preprocessing.read_dataset(self.file)

        # Given 'text', it is created two dictionaries
        # a dictiornary about: from char to index
        # a dictorionary about: from index to char
        self.char_to_idx, self.idx_to_char = preprocessing.create_dictionary(text)
        
        # Given the 'window', it is created the set of training sentences as well as
        # the set of target chars
        self.sequences, self.targets = preprocessing.build_sequences_target(text, self.char_to_idx, window=self.window)
        
        # Gets the vocabuly size
        self.vocab_size = len(self.char_to_idx)


    def train(self, args):

        # Model initialization
        model = TextGenerator(args, self.vocab_size)
        # Optimizer initialization
        optimizer = optim.RMSprop(model.parameters(), lr=self.learning_rate)
        # Defining number of batches
        num_batches = int(len(self.sequences) / self.batch_size)
        # Set model in training mode
        model.train()
        
        # Training pahse
        for epoch in range(self.num_epochs):
            # Mini batches
            for i in range(num_batches):

                # Batch definition
                try:
                    x_batch = self.sequences[i * self.batch_size : (i + 1) * self.batch_size]
                    y_batch = self.targets[i * self.batch_size : (i + 1) * self.batch_size]
                except:
                    x_batch = self.sequences[i * self.batch_size :]
                    y_batch = self.targets[i * self.batch_size :]

                # Convert numpy array into torch tensors
                x = torch.from_numpy(x_batch).type(torch.LongTensor)
                y = torch.from_numpy(y_batch).type(torch.LongTensor)
                
                # Feed the model
                y_pred = model(x)
                # Loss calculation
                loss = F.cross_entropy(y_pred, y.squeeze())
                # Clean gradients
                optimizer.zero_grad()
                # Calculate gradientes
                loss.backward()
                # Updated parameters

                optimizer.step()
        
            print("Epoch: %d,  loss: %.5f " % (epoch, loss.item()))


        torch.save(model.state_dict(), 'weights/textGenerator_model.pt')


    @staticmethod
    def generator(model, seq, idx_to_char, char_to_idx, n_chars):
        
        # Set the model in evalulation mode
        model.eval()
        
        # Define the softmax function
        softmax = nn.Softmax(dim=1)
        
        a = (char_to_idx[value] for value in seq)
        x = (tuple(a))
        #print(x)

        # The inputted text is converted to a tuple and is given to pattern
        pattern = x
        
        # By making use of the dictionaries, it is printed the pattern
        print("\nText entered:")
        print(''.join([idx_to_char[value] for value in pattern]))
        
        # In full_prediction we will save the complete prediction
        # tuple is converted to numpy array
        pattern = np.asarray(pattern)
        full_prediction = pattern
        # The prediction starts, it is going to be predicted a given
      # number of characters
        for i in range(n_chars):
        
            # The numpy patterns is transformed into a tesor-type and reshaped
            pattern = torch.from_numpy(pattern).type(torch.LongTensor)
            pattern = pattern.view(1,-1)
            
            # Make a prediction given the pattern
            prediction = model(pattern)
            # It is applied the softmax function to the predicted tensor
            prediction = softmax(prediction)
            
            # The prediction tensor is transformed into a numpy array
            prediction = prediction.squeeze().detach().numpy()
            # It is taken the idx with the highest probability
            arg_max = np.argmax(prediction)
            
            # The current pattern tensor is transformed into numpy array
            pattern = pattern.squeeze().detach().numpy()
            # The window is sliced 1 character to the right
            pattern = pattern[1:]
            # The new pattern is composed by the "old" pattern + the predicted character
            pattern = np.append(pattern, arg_max)
            
            # The full prediction is saved
            full_prediction = np.append(full_prediction, arg_max)
            
        print("\nSong prediction: ")
        print(''.join([idx_to_char[value] for value in full_prediction]) )

if __name__ == '__main__':

    args = parameter_parser()

    # If you already have the trained weights
    if args.load_model == True:
        if os.path.exists(args.model):
            
            # Load and prepare sequences
            execution = Execution(args)
            execution.prepare_data()
            
            seq = execution.seq
            idx_to_char = execution.idx_to_char
            vocab_size = execution.vocab_size
            char_to_idx = execution.char_to_idx

            # Initialize the model
            model = TextGenerator(args, vocab_size)
            
            # Load weights
            model.load_state_dict(torch.load('weights/textGenerator_model.pt'))
            seq = input("Enter the first 5 letters of the song: ")
            # Text generator
            
            execution.generator(model,seq , idx_to_char, char_to_idx,15)

            

    # If you will train the model 		
    else:
        # Load and preprare the sequences
        execution = Execution(args)
        execution.prepare_data()
        
        # Training the model
        execution.train(args)
        sequences = execution.sequences
        idx_to_char = execution.idx_to_char
        vocab_size = execution.vocab_size
        char_to_idx = execution.char_to_idx
        # Initialize the model
        model = TextGenerator(args, vocab_size)
        # Load weights
        model.load_state_dict(torch.load('weights/textGenerator_model.pt'))
        seq = input("Enter the first 5 letters of the song: ")
        # Text generator
        execution.generator(model, seq, idx_to_char, char_to_idx,15)
        
pickle.dump(model, open('model.pkl', 'wb'))
                        
if seq.lower() == 'uthra':
    print('Actual Song: Uthrada poonilaave vaa....')
    print('Youtube link: https://youtu.be/0QtNxqAZlXQ')
elif seq.lower() == 'onamn':
    print('Actual Song: Onam nilavile pole onam kinavithal pole...')
    print('Youtube link: https://youtu.be/W_lWEvwhlbU')
elif seq.lower()  == 'onamp':
    print('Actual Song: Onam ponnonam poomala pongum...')
    print('Youtube link: https://youtu.be/kxw-EO-6Z8I')
elif seq.lower() == 'onamv':
    print('Actual Song: Onam vannallo oonjaalittallo...')
    print('Youtube link: https://youtu.be/oaSMBo7FYkM')
elif seq.lower() == 'mavel':
    print('Actual Song: Maveli nadu vaneedum kalam...')
    print('Youtube link: https://youtu.be/4XsmZvalkUY')
elif seq.lower() == 'kutta':
    print('Actual Song: Kuttanadan Punjayile...')
    print('Youtube link: https://youtu.be/s2R_geXB174')
elif seq.lower() == 'Onapo':
    print('Actual Song: Onapove omal pove Ppookudiyan...')
    print('Youtube link: https://youtu.be/c0FHgOkqe1U')
else:
    print('Unable to find the song!! Please try some other song.')



Epoch: 0,  loss: 2.03803 
Epoch: 1,  loss: 1.79939 
Epoch: 2,  loss: 1.81270 
Epoch: 3,  loss: 1.65167 
Epoch: 4,  loss: 1.61717 
Epoch: 5,  loss: 1.60446 
Epoch: 6,  loss: 1.44311 
Epoch: 7,  loss: 1.49331 
Epoch: 8,  loss: 1.38388 
Epoch: 9,  loss: 1.60384 
Epoch: 10,  loss: 1.46723 
Epoch: 11,  loss: 1.47592 
Epoch: 12,  loss: 1.34274 
Epoch: 13,  loss: 1.46378 
Epoch: 14,  loss: 1.50576 
Epoch: 15,  loss: 1.27657 
Epoch: 16,  loss: 1.08034 
Epoch: 17,  loss: 1.01937 
Epoch: 18,  loss: 1.12431 
Epoch: 19,  loss: 1.18418 
Epoch: 20,  loss: 1.16331 
Epoch: 21,  loss: 0.99330 
Epoch: 22,  loss: 0.96192 
Epoch: 23,  loss: 1.00864 
Epoch: 24,  loss: 0.95013 
Epoch: 25,  loss: 1.11932 
Epoch: 26,  loss: 1.12787 
Epoch: 27,  loss: 0.99113 
Epoch: 28,  loss: 1.14227 
Epoch: 29,  loss: 1.19187 
Epoch: 30,  loss: 1.19786 
Epoch: 31,  loss: 1.05300 
Epoch: 32,  loss: 1.47006 
Epoch: 33,  loss: 1.12750 
Epoch: 34,  loss: 0.87186 
Epoch: 35,  loss: 0.99905 
Epoch: 36,  loss: 1.10778 
Epoch: 37, 

Enter the first 5 letters of the song:  onamp



Text entered:
onamp

Song prediction: 
onamponnamarana pova
Actual Song: Onam ponnonam poomala pongum...
Youtube link: https://youtu.be/kxw-EO-6Z8I


In [5]:
import speech_recognition as sr

while 1:
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        
        try:
            filename =  "draft.txt"
            f = open(filename, "a+")
            
            recognized_text = r.recognize_google(audio)
            print(recognized_text)
            remainder = recognized_text.split()
            while remainder:
                line, remainder = remainder[:12], remainder[12:]
                f.write(' '.join(line))
            if recognized_text == 'stop':
                break
                
        except sr.UnknownValueError:
            print("Unable to Understand the audio")
        except sr.RequestErrror as o:
            print("Error: (o)".format(o))
            
        break      

hello


In [6]:
from mains import Execution


with open('draft.txt', 'r') as f:
    fil = f.read()
    print(fil)
fil = fil.lower()
if len(fil) == 5 and(fil=='uthra'or fil=='onamn'or fil =='onamp'or fil =='onamv'or fil =='mavel'or fil =='kutta'or fil =='Onapo'):
    
    h = Execution(args)
    h.prepare_data()
        
    # Training the model
    h.train(args)
    sequences = h.sequences
    idx_to_char = h.idx_to_char
    vocab_size = h.vocab_size
    char_to_idx = h.char_to_idx
    # Initialize the model
    model = TextGenerator(args, vocab_size)
    h.generator(model, fil, idx_to_char, char_to_idx,15)
    

else:
    print("Please try ones again")

if fil.lower() == 'uthra':
    print('Actual Song: Uthrada poonilaave vaa....')
    print('Youtube link: https://youtu.be/0QtNxqAZlXQ')
elif fil.lower() == 'onamn':
    print('Actual Song: Onam nilavile pole onam kinavithal pole...')
    print('Youtube link: https://youtu.be/W_lWEvwhlbU')
elif fil.lower()  == 'onamp':
    print('Actual Song: Onam ponnonam poomala pongum...')
    print('Youtube link: https://youtu.be/kxw-EO-6Z8I')
elif fil.lower() == 'onamv':
    print('Actual Song: Onam vannallo oonjaalittallo...')
    print('Youtube link: https://youtu.be/oaSMBo7FYkM')
elif fil.lower() == 'mavel':
    print('Actual Song: Maveli nadu vaneedum kalam...')
    print('Youtube link: https://youtu.be/4XsmZvalkUY')
elif fil.lower() == 'kutta':
    print('Actual Song: Kuttanadan Punjayile...')
    print('Youtube link: https://youtu.be/s2R_geXB174')
elif fil.lower() == 'Onapo':
    print('Actual Song: Onapove omal pove Ppookudiyan...')
    print('Youtube link: https://youtu.be/c0FHgOkqe1U')
else:
    print('Unable to find the actual song!! Please try some other song.')
    
file = open('draft.txt', 'w')
file.close()

hello
Please try ones again
Unable to find the actual song!! Please try some other song.
